In [10]:
%reload_ext autoreload
%autoreload 2

from copy import deepcopy
from datetime import datetime, timedelta
from file_exchange import config, network
from file_exchange.network import Network
from file_exchange.peers_view import PeersView
from file_exchange.user import User
from gevent.event import AsyncResult
from gevent.pool import Group
from gevent.select import select
import gevent
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import multiprocessing
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import seaborn as sns
import shutil
import sys
import time
import traceback

def load_conf(path):
    f = open(path+'conf.json', 'r')
    conf = json.load(f)
    conf['period'] = datetime.strptime(conf['period'], "%H:%M:%S") - \
                      datetime.strptime("00:00", "%H:%M")
    return conf 

def fetch_dataframes(conf):
    devices_df = pd.DataFrame()
    users_df = pd.DataFrame()
    for fn in os.listdir(conf['output_dir']):
        if not fn.endswith(".csv"):
            continue
            
        filepath = conf['output_dir']+fn
        if re.search("device", fn):
            devices_df = append_to_df(filepath, devices_df)
        elif re.search("user", fn):
            users_df = append_to_df(filepath, users_df)

    devices_df['t'] -= min(devices_df['t'])
    devices_df = devices_df.sort_values('t')
    users_df['t'] -= min(users_df['t'])
    users_df = users_df.sort_values('t')
    return devices_df, users_df

def fetch_experiment_data(path):
    conf = load_conf(path)
    with open(conf['output_dir']+"files_info.bin", 'rb') as f:
        files_info = pickle.load(f)
    devices_df, users_df = fetch_dataframes(conf)  
    return conf, files_info, devices_df, users_df

def append_to_df(filepath, df):
    try:
        df2 = pd.read_csv(filepath)
    except:
        print("Failed loading {}: {}".format(fn, sys.exc_info()))
        pass
    else:
        if(df.empty):
            df = df2
        else:
            df = df.append(df2,)
    return df

# if experiment_set:
#     data_dir = 'data/'+experiment_set_name+''/'+conf['experiment_name']+'/'
# else:
#     data_dir = "data/"+conf['experiment_name']+'/'
# #data_dir="data/experiment_set/experiment_1/"
# conf, files_info, devices_df, users_df = fetch_experiment_data(data_dir)

In [12]:
def fetch_experiment_set(path):
    devices_df = pd.DataFrame()
    users_df = pd.DataFrame()
    conf = {}
    files_info = {}
    for experiment_name in os.listdir(path):
        conf_, files_info_, devices_df_, users_df_ = \
            fetch_experiment_data(path+experiment_name+"/")
        
        devices_df_['experiment_name'] = experiment_name
        users_df_['experiment_name'] = experiment_name
        devices_df = devices_df.append(devices_df_, ignore_index=True)
        users_df = users_df.append(users_df_, ignore_index=True)
        conf[experiment_name] = conf_
        files_info[experiment_name] = files_info_
    return conf, files_info, devices_df, users_df

conf, files_info, devices_df, users_df =  fetch_experiment_set("data/users_number/")

In [15]:
devices_df

,t,experiment_id,addr,owner,current_round,file_id,chunk_id,mess_id,type,sent,received,forwarded,success,messages_in_queue,experiment_name
0,0.000000,22,zo6gls3o4w66,Adaline,0,2zes3d,76,20289,1,612,0,0,True,0,experiment22_15users
1,0.007219,22,4gkk9i20kivj,Georgene,0,qz7lrk,14,29547,1,612,0,0,True,0,experiment22_15users
2,0.014309,22,581cyn0cux7r,Luke,0,gwsw0l,9,12996,1,612,0,0,True,0,experiment22_15users
3,0.021407,22,u2mgizf7vbnx,Luke,0,9j9mob,63,33595,1,612,0,0,True,0,experiment22_15users
4,0.028335,22,lmsqd8wpfldt,January,0,m7rdxz,55,33702,1,612,0,0,True,0,experiment22_15users
5,0.035070,22,ex8dbxkdn6gc,Delaine,0,fyrvhq,4,58719,1,612,0,0,True,0,experiment22_15users
6,0.041881,22,u4uh19q4yfyh,Maxwell,0,nzfe7z,48,4340,1,612,0,0,True,0,experiment22_15users
7,0.048827,22,knhif26buidt,Hassie,0,h7a035,23,30765,1,612,0,0,True,0,experiment22_15users
8,0.055794,22,jy15fze0a0c2,Queen,0,ceq564,74,58415,1,612,0,0,True,0,experiment22_15users
9,0.062740,22,zkmx8d8h3u7g,Queen,0,4bafv6,5,12110,1,612,0,0,True,0,experiment22_15users


## RTT

In [14]:
messages_id = np.unique(devices_df['mess_id'])
transfer_times = pd.DataFrame()
#transfer_times = np.zeros(len(messages_id), dtype=np.float64)

for i, m_id in enumerate(messages_id):
    times = [None] * 2
    for typ in [1, 2]:
        data = devices_df[(devices_df['mess_id'] == m_id) &
                          (devices_df['type'] == typ)]['t']
        
        if len(data) != 0:
            times[typ - 1] = max(data) - min(data)
            transfer_times = transfer_times.append(pd.Series({
                't': times[typ - 1], 
                'type': typ}), ignore_index=True)
            
    if times[0] is not None and times[1] is not None:
        transfer_times = transfer_times.append(pd.Series({
                't': sum(times), 
                'type': 3}), ignore_index=True)


def to_cdf(data):
    data_size = len(data)
    data_set=sorted(set(data))
    bins=np.append(data_set, data_set[-1]+1)
    counts, bin_edges = np.histogram(data, bins=bins, density=False)

    counts=counts.astype(float)/data_size

    # Find the cdf
    cdf = np.cumsum(counts)
    
    return cdf, bins

KeyboardInterrupt: 

In [ ]:
PLOT_DIR = "outputs/users_number/"
os.makedirs(PLOT_DIR)

In [ ]:
f, ax = plt.subplots()

# Plot the cdf
cdf, bins = to_cdf(transfer_times[transfer_times['type'] == 1]['t'])
ax.plot(bins[0:-1], cdf, label="Chunks")
cdf, bins = to_cdf(transfer_times[transfer_times['type'] == 2]['t'])
ax.plot(bins[0:-1], cdf, label="Acks")
cdf, bins = to_cdf(transfer_times[transfer_times['type'] == 3]['t'])
ax.plot(bins[0:-1], cdf, label="RTT")

ax.set_xlabel('Time (s)')
ax.set_xlim(0,)
ax.set_ylabel('CDF (%)')
ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:.0%}'.format(y))) 
ax.set_ylim(0,)
ax.legend()
plt.suptitle('Cumulative distribution function of messages transmission time')


sns.despine()
plt.savefig('outputs/users_number/messages_rtt_cdf.svg')
plt.show()